In [1]:
import os

import matplotlib.pyplot as plt
import modules.preprocessing as preprocess

from PIL import Image, ImageFilter
import json
import numpy as np

%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
from importlib import reload
reload(preprocess)

<module 'modules.preprocessing' from 'c:\\Alexandre\\M2 - IPP Sorbonne\\Project_fracture\\code\\modules\\preprocessing.py'>

In [3]:
# Path

folderPath = "../data/h_1p5cm/tauC_115Pa/1_CCGM_500um_590nm_52uN_115Pa-h_1p5cm_m_134g"
frameToPlot = 500

folder_rawSnapshots = os.path.join(folderPath, "snapshots_raw")
folder_processed_snapshots = os.path.join(folderPath, "snapshots_preProcessed")

assert os.path.isdir(folderPath)
assert os.path.isdir(folder_rawSnapshots), "Could not find raw snapshots"
assert os.path.isdir(folder_processed_snapshots), "Folder preprocessed should be created"

In [4]:
# Read all images

frame_bounds = [0, 550]

# Sort names
frame_number = (lambda name: int(name[50:-5]))
rawSnapshots_names = sorted([file for file in os.listdir(folder_rawSnapshots) 
                             if file[-5:]=='.tiff'], key=frame_number)

# Only keep frames in bounds
rawSnapshots_names = [name for name in rawSnapshots_names 
                      if (frame_bounds[0]<=frame_number(name)) and (frame_number(name)<=frame_bounds[1])]
rawSnapshots_path = [os.path.join(folder_rawSnapshots, name) for name in rawSnapshots_names]

rawSnapshots_names[:3]

['Basler_acA2440-75um__23178718__20240201_173049411_0000.tiff',
 'Basler_acA2440-75um__23178718__20240201_173049411_0001.tiff',
 'Basler_acA2440-75um__23178718__20240201_173049411_0002.tiff']

In [5]:
# Crop

imageRef = Image.open(rawSnapshots_path[frameToPlot])

crop_bounds = preprocess.cropBox_from_GUI(imageRef)
cropped = preprocess.crop_from_coords(imageRef, **crop_bounds)
cropped.show()

print(crop_bounds)

{'xMin': 466.75381731239463, 'yMin': 381.43424985056754, 'xMax': 1963.5532521871435, 'yMax': 1427.5245612128456}


In [6]:
# Plot histogram intensity

cropped_np = np.asarray(cropped).flatten()
std = cropped_np.std()
mean = cropped_np.mean()

std_toPlot = mean + np.arange(-3, 4) * std

fig, ax = plt.subplots()

ax.hist(cropped_np, bins=100)
for val in std_toPlot:
    ax.axvline(val, c='k', ls=':')
ax.axvline(mean, c='k', ls='-')


plt.show()

In [7]:
# Get offsets

n_std_min, n_std_max = 1.5, 5

param_contrast = preprocess.offset_scale_from_std(cropped, n_std_min=n_std_min, n_std_max=n_std_max)
contrasted = preprocess.change_intensity(cropped, *param_contrast)
contrasted.show()

In [8]:
# Add gaussian blur

radiusKernel = 0.5

blurred = preprocess.gaussian_blur(contrasted, radiusKernel)
blurred.show()

In [9]:
# Crop and contrast every image

blur = True
nbFrames = len(rawSnapshots_names)

for i in range(nbFrames):
    image = Image.open(rawSnapshots_path[i])
    cropped = preprocess.crop_from_coords(image, **crop_bounds)
    contrasted = preprocess.change_intensity(cropped, *param_contrast)
    if blur:
        blurred = preprocess.gaussian_blur(contrasted, radiusKernel)
        blurred.save(os.path.join(folder_processed_snapshots, f"processed_{rawSnapshots_names[i]}"))
    else:
        contrasted.save(os.path.join(folder_processed_snapshots, f"processed_{rawSnapshots_names[i]}"))


In [10]:
# Save data

savepath_dict = os.path.join(folder_processed_snapshots, "params_preprocess.txt")

offset_contrast, scale_contrast = param_contrast
dict_data = {"offset_contrast": offset_contrast, "scale_contrast": scale_contrast, 
             "n_std_min": n_std_min, "n_std_max": n_std_max, **crop_bounds}

with open(savepath_dict, 'w') as f:
    f.write(json.dumps(dict_data))